In [1]:
from ucimlrepo import fetch_ucirepo
from qiskit.circuit.library import ZZFeatureMap, ZFeatureMap, TwoLocal
from qiskit_machine_learning.algorithms import QSVC
from qiskit_machine_learning.algorithms import PegasosQSVC
from qiskit.primitives import Sampler
from qiskit_machine_learning.state_fidelities import ComputeUncompute
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit import QuantumCircuit
from qiskit.circuit.library import BlueprintCircuit
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from qiskit_aer import AerSimulator
from qiskit_machine_learning.optimizers import COBYLA
from qiskit_machine_learning.utils import algorithm_globals
from qiskit_machine_learning.algorithms import VQC
import matplotlib.pyplot as plt

data = 'wine.csv'
df = pd.read_csv(data)
X = df.drop(['target'], axis=1)
y = df['target']

X_array = X.to_numpy()
n_qubits = X_array.shape[1]
qc = QuantumCircuit(n_qubits)
for row in X_array:
    for i, feature in enumerate(row):
        qc.ry(feature * np.pi, i)

In [2]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
X_train.shape, X_test.shape

((142, 13), (36, 13))

In [9]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
feature_map = ZZFeatureMap(feature_dimension = X_train.shape[1], reps = 2, entanglement = 'full')
sampler = Sampler()
fidelity = ComputeUncompute(sampler=sampler)
kernel = FidelityQuantumKernel(fidelity=fidelity, feature_map=feature_map)
quantum_instance = AerSimulator(method='statevector', shots = 1024)
ansatz = TwoLocal(feature_map.num_qubits, ["ry", "rz"], "cz", reps=3)
vqc = VQC(feature_map=feature_map, ansatz=ansatz, optimizer=COBYLA(maxiter=100),)
print("Training VQC...")
vqc.fit(X_train, X_train)
score = vqc.score(X_test, y_test)
print("VQC Score:", qsvc_score)
y_pred = vqc.predict(X_test)
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

C:\Users\darry\AppData\Local\Temp\ipykernel_20156\117441539.py:5: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  sampler = Sampler()
C:\Users\darry\AppData\Local\Temp\ipykernel_20156\117441539.py:6: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  fidelity = ComputeUncompute(sampler=sampler)


Training VQC...


QiskitMachineLearningError: "Shapes don't match, predict: (142, 1069), target: (1846, 1069)!"